## Задача. Разбиение на страты

Есть набор признаков. Используя эти признаки, нужно разбить объекты на страты так, чтобы дисперсия стратифицированного среднего была минимальна и доля каждой страты была не менее 5% от всех данных.

Разбиение должно быть простым, без использования ML. Можно использовать различные логические операции типа `>, <, >=, <=, ==, |, &` и преобразования `+, -, *, /`. Например, условие для одной из страт может быть таким `(x1 == 10) & (x3 + x5 < x6)`.

Данные разбиты на две части случайным образом. Первая часть будет предоставлена для исследования. Решение будет проверяться на второй части данных. Значения в столбцах `x1, ..., x10` - признаки, которые можно использовать для вычисления страт. Значения в столбце `y` - измерения, по которым будет вычисляться целевая метрика эксперимента.

Для получения 10 баллов достаточно получить дисперсию меньше 47000. Значения дисперсии больше 47000 штрафуются с шагом 400. Если минимальная доля страты меньше 0.05, то накладывается штраф с шагом 0.001. Подробнее функцию вычисления оценки смотрите в ячейке `#тесты`

In [233]:
import pandas as pd
import numpy as np


def get_strats(df_features):
    """Возвращает страты объектов.

    :param df_features (pd.DataFrame): таблица с признаками x1,...,x10
    :return (list | np.array | pd.Series): список страт объектов размера len(df).
    """
    df_features['x11'] = df_features['x2'] + df_features['x7']
    df_features['x12'] = np.abs(df_features['x2'] - 30)
    res = np.zeros(len(df_features))
    for i in range(7):
        res[(df_features['x12'] == i).values] = i
    res[(df_features['x12'] < 9) * (df_features['x12'] >= 7)] = 7
    res[df_features['x12'] >= 9] = 8
    return res.astype(int)

In [234]:
# тесты
df = pd.read_csv('04-strat-hw-data-public.csv')
df_features = df.drop('y', axis=1)
df['strat'] = get_strats(df_features)

def calc_strat_params(df):
    """Вычисляет стратифицированную дисперсию и минимальную долю страт."""
    strat_vars = df.groupby('strat')['y'].var()
    weights = df['strat'].value_counts(normalize=True)
    stratified_var = (strat_vars * weights).sum()
    min_part = df['strat'].value_counts(normalize=True).min()
    return stratified_var, min_part

stratified_var, min_part = calc_strat_params(df)
print(f'var={stratified_var:0.0f}, min_part={min_part*100:0.2f}%')

score_ = int(np.clip(10 - np.ceil((stratified_var - 47000) / 400), 0, 10))
penalty = int(np.clip(np.ceil((0.05 - min_part) * 1000), 0, 10))
score = max(score_ - penalty, 0)
print(f'score = max({score_}-{penalty}, 0) = {score}')

var=47862, min_part=6.29%
score = max(7-0, 0) = 7
